In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import numpy as np
import matplotlib.pyplot as plt
import torchvision
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import time
import csv
import numpy as np
import math

In [2]:
df_train = pd.read_csv("df_train.csv")
df_test = pd.read_csv("df_test.csv")
print(df_train.dtypes)

Unnamed: 0                    int64
uid                           int64
age_group                     int64
gender                        int64
city                          int64
prodName                      int64
ramCapacity                 float64
ramLeftRation               float64
romCapacity                 float64
romLeftRation               float64
color                         int64
fontSize                    float64
ct                            int64
carrier                       int64
os                            int64
bootTimes                     int64
AFuncTimes                  float64
BFuncTimes                  float64
CFuncTimes                  float64
DFuncTimes                  float64
EFuncTimes                  float64
FFuncTimes                  float64
FFuncSum                      int64
appId                         int64
usage_cnt                   float64
usage_appid_cnt             float64
usage_date_cnt              float64
duration_mean               

In [3]:
print(df_test.dtypes)

Unnamed: 0                    int64
uid                           int64
gender                        int64
city                          int64
prodName                      int64
ramCapacity                 float64
ramLeftRation               float64
romCapacity                 float64
romLeftRation               float64
color                         int64
fontSize                    float64
ct                            int64
carrier                       int64
os                            int64
bootTimes                     int64
AFuncTimes                  float64
BFuncTimes                  float64
CFuncTimes                  float64
DFuncTimes                  float64
EFuncTimes                  float64
FFuncTimes                  float64
FFuncSum                      int64
appId                         int64
usage_cnt                   float64
usage_appid_cnt             float64
usage_date_cnt              float64
duration_mean               float64
duration_max                

In [4]:
class PointDataset(Dataset):
    def __init__(self, df, transform=None, train=True):
        '''
        Args:
            csv_file(string): path of label file
            transform (callable, optional): Optional transform to be applied
                on a sample.
        '''
        self.dataframe = df
        print(self.dataframe.dtypes)
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        x = self.dataframe.iloc[idx, 3:35]
        x = x.astype('float')
        x = (x - x.mean()) / (x.std())
        #print(x)
        point = np.array(x)
        label = int(self.dataframe.iloc[idx, 2])
        label = label - 1
        if self.transform is not None:
            point = self.transform(point)
        sample = {'point': point, 'label': label}
        return sample

In [30]:
class Network(nn.Module):
    def __init__(self, n_feature, n_hidden, n_output):
        '''
        Args:
            n_feature(int): size of input tensor
            n_hidden(int): size of hidden layer 
            n_output(int): size of output tensor
        '''
        super(Network, self).__init__()
        # define a liner layer
        self.hidden1 = nn.Linear(n_feature, n_hidden)
        self.hidden2 = nn.Linear(n_hidden, n_hidden)
        # define sigmoid activation 
        self.sigmoid1 = nn.Tanh()
        self.sig = nn.Sigmoid()
        self.predict = nn.Linear(n_hidden, n_output)

    def forward(self, x):
        '''
        x(tensor): inputs of the network
        '''
        # hidden layer
        h1 = self.hidden1(x)
        # activate function
        h2 = self.sigmoid1(h1)
        h3 = self.hidden2(h2)
        h4 = self.sig(h3)
        # output layer
        out = self.predict(h4)
        #out = nn.functional.log_softmax(h8) + 1e-10
        '''
        Linear classifier often follows softmax to output probability,
        however the loss function CrossEntropy we used have done this 
        operation, so we don't use softmax function here.
        '''
        return out

In [6]:
def predict(classifier_net, testloader, device):
#     correct = [0 for i in range(3)]
#     total = [0 for i in range(3)]
    correct = 0
    total = 0
    
    with torch.no_grad():
        '''
        you can also stop autograd from tracking history on Tensors with .requires_grad=True 
        by wrapping the code block in with torch.no_grad():
        '''
        for data in testloader:
            point, label = data['point'], data['label']
            point, label = point.to(device).to(torch.float32), label.to(device)
            outputs = classifier_net(point)
            '''
            if you want to get probability of the model prediction,
            you can use softmax function here to transform outputs to probability.
            '''
            # transform the prediction to one-hot form
            _, predicted = torch.max(outputs, 1)
            print('model prediction: ', predicted)
            print('ground truth:', label, '\n')
            correct += (predicted == label).sum()
            total += label.size(0)
            print('current correct is:', correct.item())
            print('current total is:', total)
            
        print('the accuracy of the model is {0:5f}'.format(correct.item()/total))
        
    return correct.item() / total

In [7]:
def clip_gradient(optimizer, grad_clip):
    """
    Clips gradients computed during backpropagation to avoid explosion of gradients.

    :param optimizer: optimizer with the gradients to be clipped
    :param grad_clip: clip value
    """
    for group in optimizer.param_groups:
        for param in group["params"]:
            if param.grad is not None:
                param.grad.data.clamp_(-grad_clip, grad_clip)

In [8]:
def show_running_loss(running_loss):
    # generate x value
    x = np.array([i for i in range(len(running_loss))])
    # generate y value
    y = np.array(running_loss)
    # define a graph
    plt.figure()
    # generate curve
    plt.plot(x, y, c='b')
    # show axis
    plt.axis()
    # define title
    plt.title('loss curve:')
    #define the name of x axis
    plt.xlabel('step')
    plt.ylabel('loss value')
    # show graph
    plt.show()

In [9]:
def predict(classifier_net, testloader, device):
#     correct = [0 for i in range(3)]
#     total = [0 for i in range(3)]
    correct = 0
    total = 0
    
    with torch.no_grad():
        '''
        you can also stop autograd from tracking history on Tensors with .requires_grad=True 
        by wrapping the code block in with torch.no_grad():
        '''
        for data in testloader:
            point, label = data['point'], data['label']
            point, label = point.to(device).to(torch.float32), label.to(device)
            outputs = classifier_net(point)
            '''
            if you want to get probability of the model prediction,
            you can use softmax function here to transform outputs to probability.
            '''
            # transform the prediction to one-hot form
            _, predicted = torch.max(outputs, 1)
            print('model prediction: ', predicted)
            print('ground truth:', label, '\n')
            correct += (predicted == label).sum()
            total += label.size(0)
            print('current correct is:', correct.item())
            print('current total is:', total)
            
        print('the accuracy of the model is {0:5f}'.format(correct.item()/total))
        
    return correct.item() / total

In [10]:
def show_accuracy(running_accuracy):
    x = np.array([i for i in range(len(running_accuracy))])
    y = np.array(running_accuracy)
    plt.figure()
    plt.plot(x, y, c='b')
    plt.axis()
    plt.title('accuracy curve:')
    plt.xlabel('step')
    plt.ylabel('accuracy value')
    plt.show()

In [11]:
def train(classifier_net, trainloader, trainframe, device, lr, optimizer):
    '''
    Args:
        classifier_net(nn.model): train model
        trainloader(torch.utils.data.DateLoader): train loader
        testloader(torch.utils.data.DateLoader): test loader
        device(torch.device): the evironment your model training
        LR(float): learning rate
    '''
    # loss function
    criterion = nn.CrossEntropyLoss().to(device)
    
    optimizer = optimizer
    
    # save the mean value of loss in an epoch
    running_loss = []
    
    running_accuracy = []
    
    # count loss in an epoch 
    temp_loss = 0.0
    
    # count the iteration number in an epoch
    iteration = 0 

    for epoch in range(epoches):
        
        '''
        adjust learning rate when you are training the model
        '''
        
        # adjust learning rate
        if epoch % 20 == 0 and epoch != 0:
            LR = LR * 0.5
            for param_group in optimizer.param_groups:
                param_group['lr'] = LR

        for i, data in enumerate(trainloader):
            point, label = data['point'], data['label']
            point, label = point.to(device).to(torch.float32), label.to(device)
            outputs = classifier_net(point)
            
            
            optimizer.zero_grad()
            loss = criterion(outputs, label)
            #if loss > 0.0 and loss < 2.0:
            loss.backward()
            
            
            if grad_clip is not None:
                clip_gradient(optimizer, grad_clip)
            optimizer.step()
            temp_loss += loss.item()
            iteration +=1 
            if iteration % 5000 == 0:
                print('[{0:d},{1:5.0f}] loss {2:.5f}'.format(epoch + 1, i, loss.item()))
            #time.sleep(1)
            #for parameters in classifier_net.parameters():
                #print(parameters)
        running_loss.append(temp_loss / iteration)
        temp_loss = 0
        iteration = 0
        print('test {}:----------------------------------------------------------------'.format(epoch))
        
        # call test function and return accuracy
        vali = trainframe.sample(n=20000, replace=False, axis=0)
        valiset = PointDataset(vali, torch.tensor)
        testloader = DataLoader(dataset=valiset, batch_size=200, shuffle=False)
        running_accuracy.append(predict(classifier_net, testloader, device))
        
        torch.save(classifier_net, './multilayersclassifier.pt')
    # show loss curve
    show_running_loss(running_loss)
    
    # show accuracy curve
    show_accuracy(running_accuracy)
    
    return classifier_net

In [24]:
batch_size = 16
transform = torch.tensor
device = torch.device('cuda:0')
trainset = PointDataset(df_train, transform=transform)

trainloader = DataLoader(dataset=trainset, batch_size=batch_size, shuffle=True)

Unnamed: 0                    int64
uid                           int64
age_group                     int64
gender                        int64
city                          int64
prodName                      int64
ramCapacity                 float64
ramLeftRation               float64
romCapacity                 float64
romLeftRation               float64
color                         int64
fontSize                    float64
ct                            int64
carrier                       int64
os                            int64
bootTimes                     int64
AFuncTimes                  float64
BFuncTimes                  float64
CFuncTimes                  float64
DFuncTimes                  float64
EFuncTimes                  float64
FFuncTimes                  float64
FFuncSum                      int64
appId                         int64
usage_cnt                   float64
usage_appid_cnt             float64
usage_date_cnt              float64
duration_mean               

In [13]:
verifyset = PointDataset(df_vali, transform=transform)
testloader = DataLoader(dataset=verifyset, batch_size=batch_size, shuffle=False)

NameError: name 'df_vali' is not defined

In [31]:
classifier_net = Network(32, 256, 6).to(device)
for parameters in classifier_net.parameters():
                print(parameters)

Parameter containing:
tensor([[-0.1113,  0.0782, -0.1074,  ...,  0.0086,  0.1703,  0.0579],
        [ 0.0433,  0.0809, -0.1644,  ..., -0.0951,  0.0011, -0.0646],
        [ 0.1299, -0.1045,  0.1538,  ..., -0.1402, -0.0451, -0.1746],
        ...,
        [ 0.1603,  0.0933,  0.0025,  ...,  0.1557, -0.1511,  0.0648],
        [ 0.0458, -0.0980,  0.0470,  ..., -0.0489, -0.1182, -0.1504],
        [ 0.1001,  0.0737,  0.0247,  ...,  0.1766,  0.0715, -0.1693]],
       device='cuda:0', requires_grad=True)
Parameter containing:
tensor([-6.1766e-02, -5.1327e-03, -1.3726e-01, -4.0150e-02, -1.3547e-01,
        -4.3849e-02, -1.5202e-01, -9.8811e-02, -7.4009e-02,  4.3065e-02,
         8.1938e-02,  2.0343e-02, -8.1226e-02, -1.4247e-01, -5.5499e-02,
        -1.2925e-01,  9.2822e-02,  1.1793e-01, -1.7618e-01, -4.5092e-03,
         4.3393e-02,  2.3148e-02,  9.8251e-02, -3.1438e-03, -1.3037e-01,
         8.9821e-02, -1.2644e-01,  8.2034e-02, -8.1748e-02,  1.3891e-02,
         4.0808e-02,  1.3736e-01,  9.375

In [32]:
lr = 0.1
device = torch.device('cuda:0')
optimizer = optim.SGD(classifier_net.parameters(), lr=lr, momentum=0.9)
optimizeradam = optim.Adam(classifier_net.parameters(), lr=lr, betas=(0.9, 0.99))
optimizerrprop = optim.RMSprop(classifier_net.parameters(), lr=lr, alpha=0.9)

In [33]:
epoches = 10
grad_clip = 5

classifier_net.to(device)

Network(
  (hidden1): Linear(in_features=32, out_features=256, bias=True)
  (hidden2): Linear(in_features=256, out_features=256, bias=True)
  (sigmoid1): Tanh()
  (sig): Sigmoid()
  (predict): Linear(in_features=256, out_features=6, bias=True)
)

In [1]:
classifier_net = train(classifier_net, trainloader, df_train, device, lr, optimizeradam)

NameError: name 'train' is not defined

In [60]:
for parameters in classifier_net.parameters():
    print(parameters)

Parameter containing:
tensor([[-0.3574, -0.6545, -0.6100,  ..., -0.4404, -0.5575, -0.5853],
        [-0.5219, -0.3637, -0.6106,  ..., -0.6108, -0.6863, -0.4711],
        [-0.6223, -0.5310, -0.4871,  ..., -0.5467, -0.4555, -0.4192],
        ...,
        [-0.8006, -0.6071, -0.5944,  ..., -0.6193, -0.5181, -0.4576],
        [-0.5060, -0.7041, -0.7436,  ..., -0.4418, -0.6517, -0.6329],
        [-0.4919, -0.6297, -0.6702,  ..., -0.5448, -0.5800, -0.6554]],
       device='cuda:0', requires_grad=True)
Parameter containing:
tensor([-0.5513, -0.5248, -0.7560, -0.7356, -0.7909, -0.6806, -0.8225, -0.5897,
        -0.7311, -0.5106, -0.6309, -0.5120, -0.8615, -0.7562, -0.7832, -0.5334,
        -0.6108, -0.5723, -0.5278, -0.7827, -0.7936, -0.7378, -0.7447, -0.5563,
        -0.7947, -0.7492, -0.5666, -0.4840, -0.5301, -0.7228, -0.5533, -0.4620,
        -0.7195, -0.6769, -0.6868, -0.5166, -0.7448, -0.5902, -0.5947, -0.7817,
        -0.5918, -0.4671, -0.5231, -0.7449, -0.7588, -0.6344, -0.6841, -0.6849

In [104]:
torch.save(classifier_net, './multilayersclassifier.pt')